In [1]:
import vcf
import pandas as pd

In [ ]:
def getInfo(vcf_file, cancer_ID, qual_flag):
    """
    Takes in a VCF file and extracts the specified columns
    vcf_file: name of the VCF file
    cancer_ID: sample ID
    qual_flag: whether to check for quality. 
               If True - filter for quality threshold at 30. 
    
    return: filtered list of dict
    """
    out = list()
    for record in vcf_file:
        if qual_flag == True and record.QUAL<=30.0:
            continue
        d = {}
        d['POS'] = record.POS # the 1- based position of the variation on the sequence
        d['REF'] = record.REF # reference base at the given position of the given reference sequence  
        d['ALT'] = record.ALT # list of alternative alleles at this position
        d['CHROM'] = record.CHROM # chromosome number on which variant is being called 
        d['GT'] = record.genotype(cancer_ID)['GT'] # genotype
        d['AD'] = record.genotype(cancer_ID)['AD'] # allelic depths for the ref and alt alleles
        d['DP'] = record.genotype(cancer_ID)['DP'] # approximate read depth
        d['GQ'] = record.genotype(cancer_ID)['GQ'] # confidence that the assigned genotype is correct 
        d['PL'] = record.genotype(cancer_ID)['PL'] # normalized, phred-scaled likelihoods for genotypes
        out.append(d)
    return out

In [ ]:
# read the vcf file for patient's tumor
vcf_reader = vcf.Reader(open('G97552.vcf', 'r')) 
# filter the VCF data
my_dict = getInfo(vcf_reader, 'OF_010116NF2_a', True)
# save the filtered data
df = pd.DataFrame(my_dict)
df.to_pickle('VCF Data_Cancer')

# read vcf file for patient's blood
vcf_reader = vcf.Reader(open('G91716.vcf', 'r'))
# filter the VCF data
my_dict = getInfo(vcf_reader, 'OF_112015SJIA_2', False)
# save the filtered data
df = pd.DataFrame(my_dict)
df.to_pickle('VCF_Data_Normal')

# read the vcf file for a reference
vcf_reader = vcf.Reader(open('G91716.vcf', 'r')) 
# filter the VCF data
my_dict = getInfo(vcf_reader, 'VB_112015SJIA_1', False)
# save the filtered data
df_r = pd.DataFrame(my_dict)
df_r.to_pickle('VCF_Data_Ref')

In [2]:
# read in saved dataframes
df_n = pd.read_pickle('VCF_Data_Normal') # patient's blood
df_c = pd.read_pickle('VCF_Data_Cancer') # patient's tumor
df_r = pd.read_pickle('VCF_Data_Ref') # reference patient's blood

# convert ALT column's content to a string
df_n.ALT = df_n.ALT.apply(lambda s: str(s[0]) if len(s)==1 else (str(s[0])+'*'))
df_c.ALT = df_c.ALT.apply(lambda s: str(s[0]) if len(s)==1 else (str(s[0])+'*'))
df_r.ALT = df_r.ALT.apply(lambda s: str(s[0]) if len(s)==1 else (str(s[0])+'*'))

In [ ]:
# select rows that are common in patient's tumor and blood genomes, based on pos and chrom
common = pd.merge(df_c, df_n, how= 'inner',on=['POS','CHROM'])
common['Test'] = common['POS'].astype(str) + common['CHROM'].astype(str)
df_c['Test'] = df_c['POS'].astype(str) + df_c['CHROM'].astype(str)

# select rows of patient's tumor genome that are not present in blood genome
cancer_only = df_c[(~df_c.Test.isin(common.Test))]
cancer_only['Test'] = cancer_only['POS'].astype(str) + cancer_only['CHROM'].astype(str)

In [ ]:
# look at the first 5 rows of the dataframe
cancer_only.head()

In [ ]:
# filter data by AD and GQ
cancer_only['AD_temp'] = cancer_only.AD.apply(lambda x: x[1])
cancer_only = cancer_only.drop(cancer_only[cancer_only.AD_temp<15].index)
cancer_only = cancer_only.drop(cancer_only[cancer_only.GQ<95].index)

# drop the temporary columns created
cancer_only = cancer_only.drop(['Test', 'AD_temp'], 1)

In [ ]:
#feature set still too big. randomly dropout columns, keep a fraction.
nb = 600.0 # number of samples to keep
cancer_only = cancer_only.sample(frac=nb/len(cancer_only))

In [ ]:
# saving clean data to .csv file
df_sampled.to_csv('VCF_clean.csv')